In [ ]:
# load_dataset("nyu-mll/glue", "mnli")
# 로 dataset을 불러오기

# 학습 때는 train split만 활용
# Validation data가 필요한 경우, train split에서 가져오기

#  trainer.train()를 통해 학습된 log가 남아있어야 함

#  Dataset의 validation_matched에 대한 성능을 출력하고, 50%를 넘겨야 함

In [ ]:
# pip install transformers datasets evaluate accelerate scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

# 데이터셋 준비

In [ ]:
import random
import evaluate
import numpy as np

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

## AutoTokenizer
다양한 tokenizer를 불러올 수 있는 클래스

In [ ]:
dataset = load_dataset("nyu-mll/glue", "mnli")
# dataset['train'][0]
train_dataset = dataset['train']
validation_matched = dataset['validation_matched']
print(len(train_dataset))

392702


## datasets - load_dataset
hugging face 의 hub에서 dataset을 다운로드받을 수 있도록 만든 함수

In [ ]:
# 데이터 tokenize한 후 dataset_tokenized 에 저장
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def preprocess_function(data):
  return tokenizer(
      data["premise"],
      data["hypothesis"],
      truncation=True
)

dataset_tokenized = train_dataset.map(preprocess_function, batched=True)
validation_tokenized = validation_matched.map(preprocess_function, batched=True)

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

## tokenizer
bert-base-cased 라는 이름의 tokenizer를 가져옴okenizer: bert-base-cased 라는 이름의 tokenizer를 가져옴
-> bert model이 이해할 수 있는 형태로 변환해줌

## preprocess_function
: 토큰화된 데이터들의 결과

## .map
각 항목에 함수를 적용하는 메소드
-> preprocess_function을 적용해 모두 토큰화함

## batched
데이터를 묶음으로 처리

In [ ]:
# train 데이터를 쪼개 training data와 validation data를 만들기
dataset_split = dataset_tokenized.train_test_split(test_size=0.2)
dataset_train, dataset_val = dataset_split['train'], dataset_split['test']

len(dataset_train), len(dataset_val)

(314161, 78541)

# 모델 구현

In [ ]:
from transformers import BertConfig

config = BertConfig()

config.hidden_size = 64
config.intermediate_size = 64
config.num_hidden_layers = 2
config.num_attention_heads = 4
config.num_labels = 3

# 모델
model = AutoModelForSequenceClassification.from_config(config)

# 학습코드

## 학습인자 정의


In [ ]:
from transformers import TrainingArguments, Trainer

# 학습인자 정의
training_args = TrainingArguments(
  output_dir = 'training1',
  num_train_epochs = 5,
  per_device_train_batch_size = 128,
  per_device_eval_batch_size = 128,
  logging_strategy = "epoch", # epoch가 끝날 때마다 기록하기
  do_train = True,
  do_eval = True,
  eval_strategy = "epoch",
  save_strategy = "epoch",
  learning_rate = 1e-3,
  load_best_model_at_end = True # validation 가장 낮은 모델
)

[Training Arguments](https://huggingface.co/docs/transformers/en/main_classes/trainer#transformers.TrainingArguments)


### batch_size
데이터를 얼마나 잘라서 학습할건지 설정

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy") # 정확도 평가하는 함수

def compute_metrics(pred): # 성능 측정을 위한 측정 수치, pred: 예측정보와 정답을 모두 포함, 각각 predictions와 labels에 저장
  predictions, labels = pred
  predictions = np.argmax(predictions, axis=1) # 열을 따라 가장 큰 수의 인덱스들 반환
  return accuracy.compute(predictions=predictions, references=labels) # 정확도


### evaluate 라이브러리
모델 성능 평가하는 함수들의 모음

# 모델 학습

In [ ]:
from transformers import EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
    # callbacks = [EarlyStoppingCallback(early_stopping_patience=1)]
)

<ipython-input-13-9ab4b9514423>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.937100,0.889379,0.589463
2,0.864600,0.861522,0.608523
3,0.802600,0.853408,0.616735
4,0.747900,0.870221,0.617843
5,0.701900,0.899942,0.614087


TrainOutput(global_step=12275, training_loss=0.8108187909718686, metrics={'train_runtime': 654.7823, 'train_samples_per_second': 2398.973, 'train_steps_per_second': 18.747, 'total_flos': 65762185210470.0, 'train_loss': 0.8108187909718686, 'epoch': 5.0})

# 모델 평가

In [ ]:
trainer.evaluate(validation_tokenized)

{'eval_loss': 0.835515022277832,
 'eval_accuracy': 0.6304635761589404,
 'eval_runtime': 5.1664,
 'eval_samples_per_second': 1899.759,
 'eval_steps_per_second': 14.904,
 'epoch': 5.0}

In [ ]:
trainer.save_model()